In [1]:
from mc import generateSurface, df, cubeVertexs, cubeCenterVertex, tracedMarchingCubes, genGrid
import meshplot as mp
import numpy as np

In [14]:
N = 128
d, edgevertexs, edgenormals, polis = cubeVertexs( N, df, surfaceThresh=0.002 )

In [15]:
cubeCenterVertex( d, edgevertexs, edgenormals, N, threshold=0.1 )
v,t = generateSurface( d, polis )

In [16]:
mp.plot(v,t)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0001633…

In [ ]:
import matplotlib.pyplot as plt
plt.hist( [ len(i['edgeVertexIndexs']) for i in d.values() ], bins=np.arange(0,12) )
plt.hist( [ i['X'].shape[0] for i in d.values() ], bins=np.arange(0,12))
plt.show()

In [ ]:
edgeVertexs =  [np.concatenate( [edgevertexs[ edge ] for edge in cubeIdx['edgeVertexIndexs'].values()] )for cubeIdx in d.values() ]
edgeNormals =  [np.concatenate( [edgenormals[ edge ] for edge in cubeIdx['edgeVertexIndexs'].values()] )for cubeIdx in d.values() ]
#edgeEigenvalues =  [np.concatenate( [eigenvals[ edge ] for edge in cubeIdx['edgeVertexIndexs'].values()] )for cubeIdx in d.values() ]

edgeVertexsUsadas =  [ cubeIdx['X']for cubeIdx in d.values() ]
edgeNormalsUsadas =  [ cubeIdx['A']for cubeIdx in d.values() ]
vertexs = np.array([ i['vertex'] for i in d.values() ] )
len(edgeVertexs)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(eigenvals)
plt.show()

In [ ]:
np.min(eigenvals), np.max(eigenvals)

In [ ]:
I = 5
p = mp.plot(edgeVertexsUsadas[I],shading={'point_size':0.17, 'point_color':'black'}, return_plot=True)
p.add_points(edgeVertexs[I],  shading={'point_size':0.13, 'point_color':'red'})
p.add_lines( edgeVertexs[I], edgeNormals[I] )
p.add_points( edgevertexs, c=eigenvals, shading={'point_size':0.1})

p.add_points( np.array( [vertexs[I]] ), shading={'point_color':'orange', 'point_size':0.3})
#p.add_points( (np.sum( edgeVertexs[I], axis=0 ) / len(edgeVertexs[I]))[None,...], shading={'point_color':'lightblue', 'point_size':0.1})

In [ ]:
v,t = tracedMarchingCubes( 16, surfaceThresh=0.0045 )
mp.plot(v,t)

In [ ]:
grid = np.concatenate( [ genGrid( 8, i ) for i in range(3) ], axis=0)

p = mp.plot(grid, shading={'point_size':0.2}, return_plot=True)
p.add_points( grid[64 + 8: 64 + 12, :], shading={'point_color':'blue','point_size':0.2})
p.add_lines( np.zeros((3,3)), np.array([[1,0,0],[0,1,0],[0,0,1]]))


In [ ]:
N = 8
x,y=np.meshgrid( np.arange(N),np.arange(N), indexing='ij' )
np.concatenate( [x[...,None], y[...,None]], axis=2 )

In [ ]:
import trimesh as tm
mesh = tm.Trimesh(v,t)
mesh.export('beetle_256.obj')